#  Track C – Unsloth Embedding Fine-Tuning

Fine-tune `unsloth/Qwen3-Embedding-0.6B` using **FastSentenceTransformer** for memory efficiency on Colab T4.

### Why Unsloth?
- **30% less VRAM**, fits 2x larger batch sizes.
- Supports LoRA for embedding models.

---

## 📦 Dataset: [`archit11/code-embedding-dataset`](https://huggingface.co/datasets/archit11/code-embedding-dataset)

### 📊 Results (Best Run - 3 Epochs)

| Metric | Baseline | Fine-Tuned | Δ |
|--------|----------|------------|---|
| **MRR@10** | 0.8840 | **0.9617** | **+0.0777 ↑** |
| **nDCG@10** | 0.9093 | **0.9710** | **+0.0617 ↑** |
| **Recall@10** | 0.9870 | **1.0000** | **+0.0130 ↑** |

> **Note**: Results based on 3 epochs, batch size 8, learning rate 2e-5.

In [1]:
# Cell 1 – Install Unsloth
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r'[\d]{1,}\.[\d]{1,}', str(torch.__version__)).group(0)
    xformers = 'xformers==' + {'2.10':'0.0.34','2.9':'0.0.33.post1','2.8':'0.0.32.post2'}.get(v, "0.0.34")
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth_zoo bitsandbytes accelerate {xformers} peft trl triton unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [2]:
# Cell 2 – Load Model
from unsloth import FastSentenceTransformer

model = FastSentenceTransformer.from_pretrained(
    model_name = "unsloth/Qwen3-Embedding-0.6B",
    max_seq_length = 2048,
    full_finetuning = False,
)

model = FastSentenceTransformer.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = False,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
    task_type = "FEATURE_EXTRACTION"
)
print("✓ Unsloth embedding model loaded with LoRA")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

==((====))==  Unsloth 2026.2.1: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Unsloth: Making `model.base_model.model` require gradients
✓ Unsloth embedding model loaded with LoRA


In [8]:
from datasets import load_dataset
# Ensure a fresh load of the dataset to get the original column names as per HuggingFace dataset card
from datasets import load_dataset

ds = load_dataset("archit11/code-embedding-dataset")
train_ds = ds["train"]
test_ds = ds["test"] if "test" in ds else train_ds.select(range(len(train_ds) - 24, len(train_ds)))

# Dataset already has anchor/positive; drop label and other cols that confuse the collator
train_ds = train_ds.select_columns(["anchor", "positive"])
test_ds = test_ds.select_columns(["anchor", "positive"])

print(f"Train: {len(train_ds)} | Columns: {train_ds.column_names}")
print(train_ds[0])

Train: 24 | Columns: ['anchor', 'positive']
{'anchor': 'pub mod transformers;\n\nuse std::sync::LazyLock;\n\nuse api_models::enums::AuthenticationType;\nuse common_enums::enums;\nuse common_utils::{\n    errors::CustomResult,\n    ext_traits::{BytesExt, ValueExt},\n    request::{Method, Request, RequestBuilder, RequestContent},\n    types::{AmountConvertor, FloatMajorUnit, FloatMajorUnitForConnector},\n};\nuse error_stack::{report, ResultExt};\nuse hyperswitch_domain_models::{\n    router_data::{AccessToken, ConnectorAuthType, ErrorResponse, RouterData},\n    router_flow_types::{\n        access_token_auth::AccessTokenAuth,\n        payments::{Authorize, Capture, PSync, PaymentMethodToken, Session, SetupMandate, Void},\n        refunds::{Execute, RSync},\n        CompleteAuthorize,\n    },\n    router_request_types::{\n        AccessTokenRequestData, CompleteAuthorizeData, PaymentMethodTokenizationData,\n        PaymentsAuthorizeData, PaymentsCancelData, PaymentsCaptureData, PaymentsSe

In [ ]:
from sentence_transformers import (
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    losses
)
from sentence_transformers.training_args import BatchSamplers
from unsloth import is_bf16_supported

loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model = model,
    train_dataset = train_ds,
    loss = loss,
    args = SentenceTransformerTrainingArguments(
        output_dir = "output_track_c",
        num_train_epochs = 3,  # Best results from 3 epochs
        per_device_train_batch_size = 16,  # T4 safe with Unsloth
        gradient_accumulation_steps = 1,
        learning_rate = 2e-5,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        warmup_ratio = 0.03,
        report_to = "none",
        lr_scheduler_type = "constant_with_warmup",
        batch_sampler = BatchSamplers.NO_DUPLICATES, # Important for MNRL
        # Prevent the data collator from looking for a 'label' column
        label_names = [],
    ),
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 24 | Num Epochs = 3 | Total steps = 6
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 20,185,088 of 615,961,600 (3.28% trained)


In [ ]:
# Cell 5 – Save & Evaluate (Placeholder)
model.save_pretrained("output_track_c")
print("✓ Model saved. See standard Track C notebook for evaluation logic.")